## 在 "批踢踢實業坊（PTT）" 網站抓股市看板上的新聞（處理單頁版）
### <p style="line-height:1.6em;" align="justify;">本範例欲爬的目標網站：[[批踢踢實業坊網站]](https://www.ptt.cc/bbs/Stock/index.html)</p>

**※ 事前準備，安裝 pyquery 套件**

**在 Anaconda 的命令列（即 Anaconda Prompt 的選項，<font color=red>但是請務必以管理員身分執行</font>）中輸入指令 "<font color=blue>conda install -c anaconda pyquery</font>" 或是 "<font color=blue>pip install pyquery</font>"**

#### <p style="line-height:1.6em;" align="justify;">PyQuery 是一個類似於 jQuery 的解析網頁工具，使用 lxml 操作 xml 和 html 文檔，它的語法和 jQuery 很像。和 XPATH 以及 Beautiful Soup 比起來，PyQuery 更加靈活，提供增加節點的 class 信息，移除某個節點，提取文本信息等功能。<br><br>PyQuery 的用法 [[1]](http://www.tastones.com/zh-tw/stackoverflow/python-language/html-parsing/pyquery/) [[2]](https://www.pyquery.org/pyquery-tutorial/) [[3]](https://pythonhosted.org/pyquery/) [[4 重要]](https://pyquery.readthedocs.io/_/downloads/en/latest/pdf/) [[5 入門]](https://kknews.cc/zh-tw/code/6qqlppp.html) [[6*]](https://www.cnblogs.com/chenyangqit/p/16594908.html) [[7*]](https://github.com/leVirve/CrawlerTutorial)</p>

### 以下在 Conda 上 install PyQuery 的方式二選一 [[1]](https://gist.github.com/yassineAlouini/bf1a4597ee4ae85f5925ecc39e01e4a4) [[官網]](https://anaconda.org/anaconda/pyquery)

In [ ]:
# Install Method 1 : 
# Install a pip package in the current Jupyter Kernel

import sys

!{sys.executable} -m pip install pyquery

In [ ]:
# Install Method 2 : 
# Install a conda package in the current Jupyter Kernel

#import sys
#!conda install  --yes --prefix {sys.prefix} pyquery 

!conda install -c anaconda pyquery

In [ ]:
# 可以去掉 Python 輸出時，因為軟體版本所引起的警告的警告。

import warnings

warnings.filterwarnings('ignore')

In [ ]:
from pyquery import PyQuery as pq

from collections import OrderedDict   # 會對 key 輸入先後順序做排序的字典結構

import pandas as pd   # 當成 Python DataScience 的 Excel

In [ ]:
#
# 開始資料擷取的爬蟲動作
#

import requests

#===========================================================
#
#            以 下 的 目 標 網 址 寫 法 請 二 選 一
#
#===========================================================
#
# 方法一： 假設要爬取最新頁面的 "前" 一頁
#
# 注意！注意！
# index 後的數字在 PPT 網站上隨時會更改，請找最新看板的前一頁(標示"上頁")
# 的頁碼數字，再更新以下的目標網址中的 "indexABCD.html"
# 請注意：index<ABCD>.html 中的 ABCD 要隨時調整成適當的頁碼編號
#

# 每一次要改與更新 URL address，本例是 2025/06/19 的 PTT Stock 最後倒數第二頁頁碼
url = 'https://www.ptt.cc/bbs/Stock/index8944.html'


#
# 方法二： 與上一例相反，如果只要爬取最新的一頁，就開啟以下的網址 
#

#url = 'https://www.ptt.cc/bbs/Stock/index.html'


#
# 直接對網站內容進行解析，並將結果傳回來
#

q = pq(requests.get(url).text)   # requests.get() 方法不可以省

#### HTML Document Object Model [[1*]](https://ithelp.ithome.com.tw/articles/10202689) [[2]](https://medium.com/vicky-notes/dom-%E9%81%B8%E5%8F%96%E7%B6%B2%E9%A0%81%E5%85%83%E7%B4%A0-2c338afd238f) [[3]](https://www.w3schools.com/whatis/whatis_htmldom.asp) [[4 重要]](https://www.runoob.com/htmldom/htmldom-tutorial.html)

In [ ]:
#
# 經過解析後的 q 就是網站的 html DOM(Document Object Model) Tree
#
print(q)

In [ ]:
#
# 獲取本頁所有標題 作者 推文數資訊
#
title = q('.title a').text()

author = q('.author').text()

push = q('.hl.f3').text()

push = q('.hl').text()

In [ ]:
print(title)

In [ ]:
print(author)

In [ ]:
print(push)

**很明顯這樣的資訊並不是我們要的，因為所有文章的訊息都擠在一起了，所以接下來我們要針對每一篇文章分別爬資訊**

In [ ]:
#
# 練習一下 - 先取 q('.r-ent') 中第一個 element 區塊來孰悉指令
#

info = []

article = q('.r-ent')[0]    # 針對本業的第一篇推文來解析

print('\n如果沒有重新解析:', article)

print('\n重新解析後:', pq(article))

#### <p style="line-height:1.6em;" align="justify;"> OrderedDict（）[[1]](https://ithelp.ithome.com.tw/articles/10193794) [[2]](https://www.796t.com/content/1545492546.html)；OrderedDict 是 dict 的一個子類別，它具有一個很實用的特性就是他會對 key 做排序，而這個排序的依據在於這個 key 被插入的先後順序，而如果更新了某個 key 的value 值並不會影響他在 OrderedDict 中的排序位置，除非這個 key 被刪除並被重新插入，才會從原始的排序位置變成最末位，因為其變成最新插入的一個 key。</p>

In [ ]:
#
# 獲取每篇文章分別的標題 作者 推文數等資訊
#
info = []

for article in q('.r-ent'):
    
    #
    # 按照 Key 的存放次序（title->author->push）排列的字典型別
    #
    info_dict = OrderedDict()

    #
    # 每一篇文章的資訊分別塞到字典裡頭
    #
    article_q = pq(article)    # 記得要重新用 pq() 再解析一次
    
    info_dict['title'] = article_q('.title a').text()      # 文章標題
    
    info_dict['author'] = article_q('.author').text()      # 作者
    
    info_dict['push'] = article_q('.hl').text()            # 推文數量

    #
    # 將字典存入 list
    #
    info.append(info_dict)

In [ ]:
info    # 看一下內容

In [ ]:
#
# 將 list 轉成 pandas dataframe
#
data = pd.DataFrame(info)

#
# 文章索引值改成由 1 開始
#
data.index += 1

#
# 將撈出的結果存檔
#
data.to_csv('ptt_stock_info.csv', encoding = 'utf-8', index = None)

In [ ]:
data.head()    # 看一下前五筆的內容

**現在已經有了每篇文章個別的標題、作者、推文數資訊，那如果想知道每篇文章的推文全文內容該怎麼做呢?**

In [ ]:
#
# 獲取每篇文章的 URL
#
url_list = []

for node in q('.title a'):
    
    #
    # 十分重要：記得要重新用 pq 再解析一次
    #
    url = 'https://www.ptt.cc' + str(pq(node).attr('href'))  
    
    #
    # 加入到 URL list
    #
    url_list.append(url)

In [ ]:
url_list    # 看一下內容

In [ ]:
#
# 進入每篇文章的 URL 目標網址內以獲得全文的詳細資訊
#
page_list = []

for page in url_list:

    page_dict = OrderedDict()

    #
    # 直接對網站內容進行解析，並將結果傳回來
    #
    page_q = pq(requests.get(page).text) 
    
    # 擷取作者資訊
    page_dict['author'] = page_q('.article-metaline:nth-child(1) .article-meta-value').text()
    
    #
    # 擷取所有讀者評論訊息內容
    #
    # step 1: 找到字串中的冒號做切割點，切完後的結果是存放至一個串列中
    tmp_list1 = page_q('.push-content').text().split(':')   
    
    # step 2: 刪除串列中第一個物件
    del tmp_list1[0] 
    
    # step 3: 刪除將每一個串列內字串物件的前後空白字元
    tmp_list2 = []
    
    for obj in tmp_list1:
            
            tmp_list2.append(obj.strip())
            
    
    # step 4: 將 tmp_list 剩餘的串列內容存入 page_dict 之中
    page_dict['push_content'] = tmp_list2

    page_list.append(page_dict)

In [ ]:
page_list[0]    # 看一下第一篇文章的內容

In [ ]:
#
# 將 list 轉成 Pandas dataframe
#
page_data = pd.DataFrame(page_list)

#
# 文章索引值改成由 1 開始
#
page_data.index += 1

#
# 將所有文章存入一個 CSV 檔案內做後續文字探勘或情緒分析處理
#
page_data.to_csv('ptt_stock_pageinfo.csv', encoding = 'utf-8', index = None)

In [ ]:
page_data.head()    # 看一下前五筆的內容